In [1]:
import numpy as np
import pyvista as pv

import geogen.model as geo
import geogen.generation as gen
import geogen.plot as geovis
from geogen.probability import SedimentBuilder


pv.set_jupyter_backend('static')
WS = (600,400) # Set a custom window size to be reused in most plots


## GeoProcess Icon Sketchbook

Source code for generating examples of geoprocess for paper

### Custom Plot Type for GeoProcess Icon Sketchbook 

In [2]:
SCALING = 10.0

def demoview(model, threshold=-0.5, opacity=1, plotter=None):

    if plotter is None:
        plotter = pv.Plotter(window_size=WS)
    p=plotter
    
    cats = np.arange(15)   # Set the number of categories for the colormap
    clim = [cats.min(), cats.max()] # Preset color limits for all subplots
    
    grid = geovis.get_voxel_grid_from_model(model, threshold=threshold)
    skin=grid.extract_surface()
    
    p.add_mesh(skin,scalars='values',clim=clim,cmap='gist_ncar',opacity=opacity)
    # p.add_axes(line_width=5)    
    p.remove_scalar_bar()
    
    # flat_bounds = [item for sublist in model.bounds for item in sublist]
    # bounding_box = pv.Box(flat_bounds)
    bounding_box = get_trimmed_bounding_box()
    p.add_mesh(bounding_box, color="black", style="wireframe", line_width=(3/2)*SCALING, name="bounding_box")
    p.camera.elevation = -20
    # Zoom in to fill box
    p.camera.zoom(1.35)
    return p

def get_trimmed_bounding_box():
    new_points = np.array([
    [-20, -20, -10],
    [ 20, -20, -10],
    [-20,  20, -10],
    [ 20,  20, -10],
    [-20, -20,  10],
    [ 20, -20,  10],
    [-20,  20,  10],
])
    lines = [
        2, 0, 1,  # Line from point 0 to point 1
        2, 0, 2,  # Line from point 0 to point 2
        2, 0, 4,  # Line from point 0 to point 4
        2, 1, 3,  # Line from point 1 to point 3
        2, 1, 5,  # Line from point 1 to point 5
        2, 2, 3,  # Line from point 2 to point 3
        2, 2, 6,  # Line from point 2 to point 6
        2, 4, 5,  # Line from point 4 to point 5
        2, 4, 6   # Line from point 4 to point 6
    ]
    # Create a new PolyData for the bounding box
    line_polydata = pv.PolyData()
    line_polydata.points = new_points
    line_polydata.lines = lines
    return line_polydata

def slice_volume(model, start, stop):
    # convert to theta coords
    theta = np.arctan2(model.xyz[:,1], model.xyz[:,0])
    mask = (theta >= start) & (theta <= stop)
    model.data[mask] = -1
    return model

### Paper Images for Sample Transforms

In [3]:
bounds = ((-20, 20), (-20, 20), (-10, 10))
resolution = (300,300,150)
model = geo.GeoModel(bounds=bounds, resolution=resolution)

#### Init Strata

In [4]:
# Make a model with sediment layers
value_list = [0, 2, 4, 6, 4, 3]
thickness_list = [4, 2, 2, 3, 2, 2]
base = -10
sediment = geo.Sedimentation(value_list=value_list, thickness_list=thickness_list, base=base)

model.clear_history()
model.add_history(sediment)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1.0)
p.screenshot("images/base_strata.png", scale=SCALING, transparent_background=True);


In [5]:
def cross_layers():
    bedrock = geo.Bedrock(base=-5, value=0)
    vals = [1,2,3,4,1,2,3,4]
    thicks = [1.5]
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    return [bedrock, sediment]

def cross_layers():
    value_list = [2, 4, 6, 4, 2]
    thickness_list = [2, 2, 3, 2, 2]
    base = -6
    bedrock = geo.Bedrock(base=base, value=0)
    sediment = geo.Sedimentation(value_list=value_list, thickness_list=thickness_list, base=base)
    return [bedrock, sediment]
    

In [6]:
names = ["fold", "dike", "fault"]
strata = cross_layers()
fold = geo.Fold(strike=45, dip=90, rake=45, origin=(0, 0, 0), amplitude=3, period=20, shape=.1)
fault = geo.Fault(strike=-90, dip=45, rake=45, origin=(0, 0, 0), amplitude=8)
dike = geo.DikePlane(strike=10, dip=90, width=3, origin=(5, 0, 0), value=13)
processes = [fold, dike, fault]

### Sequential Showcase

In [7]:
model.clear_history()
model.add_history(strata)
model.compute_model()
p = demoview(model, opacity=1)
p.screenshot("images/base_strata.png", scale=SCALING, transparent_background=False);


for name, process in zip(names, processes):
    model.clear_data()
    model.add_history(process)
    model.compute_model()
    # model = slice_volume(model, 0, 2*np.pi/4)
    p = demoview(model, opacity=1)
    p.screenshot(f"images/{name}_sequential.png", scale=SCALING, transparent_background=False);


### Non-Sequential Showcase

In [8]:

for name, process in zip(names, processes):
    model.clear_history()
    model.add_history(strata)
    model.add_history(process)
    model.compute_model()
    # model = slice_volume(model, 0, 2*np.pi/4)
    p = demoview(model, opacity=1)
    p.screenshot(f"images/{name}_single.png", scale=SCALING, transparent_background=False);